    *************************** CASE STUDY HEALTHCARE DATA ANALYSIS (PART 1) ***********************************

    PART 1: Identify providers that overcharge for certain procedures or regions where procedures are too expensive

    #1. Import libraries and input

In [1]:
import pandas as pd
import numpy as np

In [2]:
Medicare_Charge_In = pd.read_csv('Medicare_Charge_Inpatient_DRG100_DRG_Summary_by_DRG_FY2011.csv')
Medicare_Charge_Out = pd.read_csv('Medicare_Charge_Outpatient_APC30_Summary_by_APC_CY2011.csv')
Medicare_Provider_Charge_In = pd.read_csv('Medicare_Provider_Charge_Inpatient_DRG100_FY2011.csv')
Medicare_Provider_Charge_Out = pd.read_csv('Medicare_Provider_Charge_Outpatient_APC30_CY2011_v2.csv')
Patient_history = pd.read_csv('Patient_history_samp.csv')
Review_patient = pd.read_csv('Review_patient_history_samp.csv')
Rreview_transaction = pd.read_csv('Rreview_transaction_coo.csv')
Transaction= pd.read_csv('Transaction_coo.csv')

    #2. Data Analysis and Data Preparation

    The Data Analysis will be based on financial summary data from 2011 that were made availible by the centers of  Medicare and Medicaid Services
    
    Medicare_Provider_Charge_Inpatient_DRG100_FY2011.csv
    Medicare_Provider_Charge_Outpatient_APC30_CY2011_v2.csv

In [3]:
Medicare_Provider_Charge_In.isnull().any()

DRG Definition                                False
Provider Id                                   False
Provider Name                                 False
Provider Street Address                       False
Provider City                                 False
Provider State                                False
Provider Zip Code                             False
Hospital Referral Region (HRR) Description    False
Total Discharges                              False
Average Covered Charges                       False
Average Total Payments                        False
Average Medicare Payments                     False
dtype: bool

In [4]:
Medicare_Provider_Charge_Out.isnull().any()

APC                                           False
Provider Id                                   False
Provider Name                                 False
Provider Street Address                       False
Provider City                                 False
Provider State                                False
Provider Zip Code                             False
Hospital Referral Region (HRR) Description    False
Outpatient Services                           False
Average  Estimated Submitted Charges          False
Average Total Payments                        False
dtype: bool

    #3. PART1A - Highest Cost Variation

    The Highest cost variation can be calculated by dividing the standard deviation by the mean.
    
    The cost variation of OutpatientsAPCs overtakes the cost variation of InpatientsDRG.
    The 3 Highest cost variation (not the highest cost of procedure) can be seen in Outpatients for follwing APCS-
    
    0604 - Level 1 Hospital Clinic Visits
    0698 - Level II Eye Tests & Treatments
    0019 - Level I Excision/ Biopsy

In [5]:
# Finding Standard deviation and mean for InpatientDRG
Medicare_IN = pd.concat(
[Medicare_Provider_Charge_In[['DRG Definition','Average Covered Charges']].groupby(['DRG Definition']).agg('std').add_suffix('_std'),
 Medicare_Provider_Charge_In[['DRG Definition','Average Covered Charges']].groupby(['DRG Definition']).agg('mean').add_suffix('_avg')]
 , axis =1)

In [6]:
#Coefficient of Variance, the standard deviation divided by the mean
Medicare_IN['Variation'] = Medicare_IN.apply(lambda x: x['Average Covered Charges_std']/x['Average Covered Charges_avg'],axis =1)

In [7]:
Medicare_IN['Variation'].nlargest(3)

DRG Definition
885 - PSYCHOSES                                                              0.717796
897 - ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REHABILITATION THERAPY W/O MCC    0.633967
917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC                               0.613150
Name: Variation, dtype: float64

In [8]:
# Finding Standard deviation and mean for OutpatientDRG
Medicare_OUT = pd.concat(
[Medicare_Provider_Charge_Out[["APC","Average  Estimated Submitted Charges"]].groupby(['APC']).agg('std').add_suffix('_std'),
Medicare_Provider_Charge_Out[["APC","Average  Estimated Submitted Charges"]].groupby(['APC']).agg('mean').add_suffix('_avg')],
axis =1)

In [9]:
#Coefficient of Variance, the standard deviation divided by the mean
Medicare_OUT['Variation'] = Medicare_OUT.apply(lambda x: x['Average  Estimated Submitted Charges_std']/x['Average  Estimated Submitted Charges_avg'],axis =1)

In [10]:
Medicare_OUT['Variation'].nlargest(3)

APC
0604 - Level 1 Hospital Clinic Visits     1.385150
0698 - Level II Eye Tests & Treatments    1.310444
0019 - Level I Excision/ Biopsy           1.229897
Name: Variation, dtype: float64

    #4. PART1B - Highest Cost Claims by provider

    The cost claims of InPatientsDRG visbibly overtakes the cost claims of OutpatientsAPC.
    The 3 Highest cost claims can be seen for follwing Providers-
    
    BAYONNE HOSPITAL CENTER       - NJ - Newark
    CROZER CHESTER MEDICAL CENTER - PA - Philadelphia
    STANFORD HOSPITAL             - CA - San Mateo County
    

#### FOR InpatientDRG

In [11]:
Medicare_Provider_Charge_In.columns

Index([u'DRG Definition', u'Provider Id', u'Provider Name',
       u'Provider Street Address', u'Provider City', u'Provider State',
       u'Provider Zip Code', u'Hospital Referral Region (HRR) Description',
       u'Total Discharges', u'Average Covered Charges',
       u'Average Total Payments', u'Average Medicare Payments'],
      dtype='object')

In [12]:
#Finding the max of Average Covered Charges for InpatientDRG
Medicare_IN2 = pd.DataFrame(Medicare_Provider_Charge_In[['DRG Definition', 'Average Covered Charges']].groupby(['DRG Definition']).agg('max').add_suffix('_max'))

In [13]:
Medicare_IN2 = Medicare_IN2.reset_index()

In [14]:
#Merging the max value with InpatientDRG dataset
Medicare_IN_MERGE = Medicare_Provider_Charge_In.merge(Medicare_IN2,on='DRG Definition', how='left')

In [15]:
Medicare_IN_MERGE.to_csv('Medicare_IN_MERGE.csv')

In [16]:
#function to get the max indicator
def difftyp(Medicare_IN_MERGE):
    if ((Medicare_IN_MERGE['Average Covered Charges_max']  == Medicare_IN_MERGE['Average Covered Charges'])):
        return 1
    else:
        return 0

In [17]:
Medicare_IN_MERGE['MAX'] = Medicare_IN_MERGE.apply(difftyp, axis=1)

In [18]:
#getting number of times a provider has charge max value for procedure
Medicare_IN_MAXCALC_PROVIDER= pd.DataFrame(Medicare_IN_MERGE[['Provider Name','MAX']].groupby(['Provider Name']).agg('sum').add_suffix('_sum'),)

In [19]:
#getting number of times a region has charge max value for procedure
Medicare_IN_MAXCALC_REGION= pd.DataFrame(Medicare_IN_MERGE[['Hospital Referral Region (HRR) Description','MAX']].groupby(['Hospital Referral Region (HRR) Description']).agg('sum').add_suffix('_sum'),)

In [20]:
Medicare_IN_MAXCALC_PROVIDER['MAX_sum'].nlargest(3)

Provider Name
BAYONNE HOSPITAL CENTER          23
CROZER CHESTER MEDICAL CENTER    11
STANFORD HOSPITAL                10
Name: MAX_sum, dtype: int64

In [105]:
Medicare_IN_MAXCALC_REGION['MAX_sum'].nlargest(5)

Hospital Referral Region (HRR) Description
NJ - Newark                 25
PA - Philadelphia           23
CA - San Mateo County       13
CA - Contra Costa County     6
CA - Alameda County          5
Name: MAX_sum, dtype: int64

#### FOR OutpatientAPC

In [22]:
Medicare_Provider_Charge_Out.columns

Index([u'APC', u'Provider Id', u'Provider Name', u'Provider Street Address',
       u'Provider City', u'Provider State', u'Provider Zip Code',
       u'Hospital Referral Region (HRR) Description', u'Outpatient Services',
       u'Average  Estimated Submitted Charges', u'Average Total Payments'],
      dtype='object')

In [23]:
#Finding the max of Average Covered Charges for OutPatientAPC
Medicare_OUT2 = pd.DataFrame(Medicare_Provider_Charge_Out[['APC', 'Average  Estimated Submitted Charges']].groupby(['APC']).agg('max').add_suffix('_max'))

In [24]:
Medicare_OUT2 = Medicare_OUT2.reset_index()

In [25]:
#Merging the max value with InpatientDRG dataset
Medicare_OUT_MERGE = Medicare_Provider_Charge_Out.merge(Medicare_OUT2,on='APC', how='left')

In [26]:
#function to get the max indicator
def difftyp(Medicare_OUT_MERGE):
    if ((Medicare_OUT_MERGE['Average  Estimated Submitted Charges_max']  == Medicare_OUT_MERGE['Average  Estimated Submitted Charges'])):
        return 1
    else:
        return 0

In [27]:
Medicare_OUT_MERGE['MAX'] = Medicare_OUT_MERGE.apply(difftyp, axis=1)

In [28]:
#getting number of times a provider has charge max value for procedure
Medicare_OUT_MAXCALC_PROVIDER= pd.DataFrame(Medicare_OUT_MERGE[['Provider Name','MAX']].groupby(['Provider Name']).agg('sum').add_suffix('_sum'),)

In [29]:
#getting number of times a region has charge max value for procedure
Medicare_OUT_MAXCALC_REGION= pd.DataFrame(Medicare_OUT_MERGE[['Hospital Referral Region (HRR) Description','MAX']].groupby(['Hospital Referral Region (HRR) Description']).agg('sum').add_suffix('_sum'),)

In [106]:
Medicare_OUT_MAXCALC_PROVIDER['MAX_sum'].nlargest(5)

Provider Name
CAPITAL HEALTH MEDICAL CENTER - HOPEWELL    2
COLUMBIA HOSPITAL                           2
FORT WALTON BEACH MEDICAL CENTER            2
BEVERLY HOSPITAL CORPORATION                1
CENTINELA HOSPITAL MEDICAL CENTER           1
Name: MAX_sum, dtype: int64

In [31]:
Medicare_OUT_MAXCALC_REGION['MAX_sum'].nlargest(3)

Hospital Referral Region (HRR) Description
PA - Philadelphia     5
CA - Los Angeles      3
CA - Orange County    2
Name: MAX_sum, dtype: int64

    #4. PART1C - Highest Cost Claims by Region

    Combining the counts of Inpatient and Outpatients Following are the 3 Highest cost claims by Region -
    
    Hospital Referral Region (HRR) Description
    CA - Contra Costa County
    CA - San Mateo County
    CA - Santa Cruz

#### FOR InpatientDRG

In [51]:
#Finding the mean of Average Covered Charges for InpatientDRG by DRG Defination and Region
Medicare_IN_AGG = pd.DataFrame(Medicare_Provider_Charge_In[['DRG Definition', 'Hospital Referral Region (HRR) Description',  'Average Covered Charges']].groupby(['DRG Definition', 'Hospital Referral Region (HRR) Description']).agg('mean').add_suffix('_avg'))

In [52]:
Medicare_IN_AGG  = Medicare_IN_AGG.reset_index()

In [53]:
Medicare_IN_AGG.head()

,DRG Definition,Hospital Referral Region (HRR) Description,Average Covered Charges_avg
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AK - Anchorage,34805.130430
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,40237.361538
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Dothan,36285.182905
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Huntsville,18878.955554
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Mobile,22802.270118


In [54]:
#Finding the max for InpatientDRG DRG Definition
Medicare_IN_MAX = pd.DataFrame(Medicare_IN_AGG[['DRG Definition', 'Average Covered Charges_avg']].groupby(['DRG Definition']).agg('max').add_suffix('_max'))

In [55]:
Medicare_IN_MAX = Medicare_IN_MAX.reset_index()

In [56]:
Medicare_IN_MAX.head()

,DRG Definition,Average Covered Charges_avg_max
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,105929.470600
1,057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/...,148128.437500
2,064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,145720.057518
3,065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,73820.938782
4,066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,65418.983067


In [57]:
#Merging The mean and max - will give side by side comparison of region having highest claims
Medicare_IN_MERGE = Medicare_IN_AGG.merge(Medicare_IN_MAX,on='DRG Definition', how='left')

In [58]:
Medicare_IN_MERGE.head()

,DRG Definition,Hospital Referral Region (HRR) Description,Average Covered Charges_avg,Average Covered Charges_avg_max
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AK - Anchorage,34805.130430,105929.4706
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,40237.361538,105929.4706
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Dothan,36285.182905,105929.4706
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Huntsville,18878.955554,105929.4706
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Mobile,22802.270118,105929.4706


In [59]:
#function to get the max indicator
def difftyp(Medicare_IN_MERGE):
    if ((Medicare_IN_MERGE['Average Covered Charges_avg'] == Medicare_IN_MERGE['Average Covered Charges_avg_max'])):
        return 1
    elif ((Medicare_IN_MERGE['Average Covered Charges_avg'] > Medicare_IN_MERGE['Average Covered Charges_avg_max'])):
        return 1
    else:
        return 0

In [60]:
Medicare_IN_MERGE['MAX'] = Medicare_IN_MERGE.apply(difftyp, axis=1)

In [61]:
Medicare_IN_MERGE[Medicare_IN_MERGE['MAX'] == 1].head()

,DRG Definition,Hospital Referral Region (HRR) Description,Average Covered Charges_avg,Average Covered Charges_avg_max,MAX
22,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,CA - Napa,105929.470600,105929.470600,1
330,057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/...,CA - San Mateo County,148128.437500,148128.437500,1
588,064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,CA - Contra Costa County,145720.057518,145720.057518,1
907,065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,CA - San Mateo County,73820.938782,73820.938782,1
1198,066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,CA - Contra Costa County,65418.983067,65418.983067,1


In [62]:
#getting number of times a region has charge max value for procedure
Medicare_IN_MERGE_GROUPED = pd.DataFrame(Medicare_IN_MERGE[['Hospital Referral Region (HRR) Description','MAX']].groupby(['Hospital Referral Region (HRR) Description']).agg('sum').add_suffix('_sum'),)

In [96]:
Medicare_IN_MERGE_GROUPED = Medicare_IN_MERGE_GROUPED.reset_index()

In [97]:
Medicare_IN_MERGE_GROUPED.head()

,Hospital Referral Region (HRR) Description,MAX_sum
0,AK - Anchorage,0
1,AL - Birmingham,0
2,AL - Dothan,0
3,AL - Huntsville,0
4,AL - Mobile,0


#### FOR OutpatientAPC

In [66]:
Medicare_Provider_Charge_Out.columns

Index([u'APC', u'Provider Id', u'Provider Name', u'Provider Street Address',
       u'Provider City', u'Provider State', u'Provider Zip Code',
       u'Hospital Referral Region (HRR) Description', u'Outpatient Services',
       u'Average  Estimated Submitted Charges', u'Average Total Payments'],
      dtype='object')

In [69]:
#Finding the mean of Average Covered Charges for Outpatient by APC and Region
Medicare_Out_AGG = pd.DataFrame(Medicare_Provider_Charge_Out[['APC', 'Hospital Referral Region (HRR) Description', 'Average  Estimated Submitted Charges']].groupby(['APC', 'Hospital Referral Region (HRR) Description']).agg('mean').add_suffix('_avg'))

In [70]:
Medicare_Out_AGG = Medicare_Out_AGG.reset_index()

In [71]:
Medicare_Out_AGG.head()

,APC,Hospital Referral Region (HRR) Description,Average Estimated Submitted Charges_avg
0,0012 - Level I Debridement & Destruction,AK - Anchorage,149.589749
1,0012 - Level I Debridement & Destruction,AL - Birmingham,78.086957
2,0012 - Level I Debridement & Destruction,AR - Fort Smith,80.102439
3,0012 - Level I Debridement & Destruction,AR - Jonesboro,16.993333
4,0012 - Level I Debridement & Destruction,AR - Little Rock,96.968863


In [72]:
#Finding the max for Outpatient
Medicare_Out_MAX = pd.DataFrame(Medicare_Out_AGG[['APC', 'Average  Estimated Submitted Charges_avg']].groupby(['APC']).agg('max').add_suffix('_max'))

In [73]:
Medicare_Out_MAX = Medicare_Out_MAX.reset_index()

In [74]:
Medicare_Out_MAX.head()

,APC,Average Estimated Submitted Charges_avg_max
0,0012 - Level I Debridement & Destruction,438.307045
1,0013 - Level II Debridement & Destruction,1042.707451
2,0015 - Level III Debridement & Destruction,1478.536545
3,0019 - Level I Excision/ Biopsy,15946.325655
4,0020 - Level II Excision/ Biopsy,15784.975000


In [75]:
#Merging The mean and max - will give side by side comparison of region having highest claims
Medicare_Out_MERGE = Medicare_Out_AGG.merge(Medicare_Out_MAX,on='APC', how='left')

In [77]:
Medicare_Out_MERGE.columns

Index([u'APC', u'Hospital Referral Region (HRR) Description',
       u'Average  Estimated Submitted Charges_avg',
       u'Average  Estimated Submitted Charges_avg_max'],
      dtype='object')

In [82]:
#function to get the max indicator
def difftyp(Medicare_Out_MERGE):
    if ((Medicare_Out_MERGE['Average  Estimated Submitted Charges_avg'] == Medicare_Out_MERGE['Average  Estimated Submitted Charges_avg_max'])):
        return 1
    elif ((Medicare_Out_MERGE['Average  Estimated Submitted Charges_avg'] > Medicare_Out_MERGE['Average  Estimated Submitted Charges_avg_max'])):
        return 1
    else:
        return 0

In [83]:
Medicare_Out_MERGE['MAX'] = Medicare_Out_MERGE.apply(difftyp, axis=1)

In [84]:
Medicare_Out_MERGE[Medicare_Out_MERGE['MAX'] == 1].head()

,APC,Hospital Referral Region (HRR) Description,Average Estimated Submitted Charges_avg,Average Estimated Submitted Charges_avg_max,MAX
93,0012 - Level I Debridement & Destruction,NE - Omaha,438.307045,438.307045,1
160,0013 - Level II Debridement & Destruction,AL - Dothan,1042.707451,1042.707451,1
582,0015 - Level III Debridement & Destruction,MI - Dearborn,1478.536545,1478.536545,1
756,0019 - Level I Excision/ Biopsy,CA - Los Angeles,15946.325655,15946.325655,1
1013,0020 - Level II Excision/ Biopsy,IL - Aurora,15784.975000,15784.975000,1


In [85]:
#getting number of times a region has charge max value for procedure
Medicare_Out_MERGE_GROUPED = pd.DataFrame(Medicare_Out_MERGE[['Hospital Referral Region (HRR) Description','MAX']].groupby(['Hospital Referral Region (HRR) Description']).agg('sum').add_suffix('_sum'),)

In [93]:
Medicare_Out_MERGE_GROUPED = Medicare_Out_MERGE_GROUPED.reset_index()

In [95]:
Medicare_Out_MERGE_GROUPED.head()

,Hospital Referral Region (HRR) Description,MAX_sum
0,AK - Anchorage,0
1,AL - Birmingham,0
2,AL - Dothan,1
3,AL - Huntsville,1
4,AL - Mobile,0


In [98]:
frames = [Medicare_IN_MERGE_GROUPED, Medicare_Out_MERGE_GROUPED]
Medicare_data_REGION = pd.concat(frames)

In [99]:
Medicare_data_REGION.head()

,Hospital Referral Region (HRR) Description,MAX_sum
0,AK - Anchorage,0
1,AL - Birmingham,0
2,AL - Dothan,0
3,AL - Huntsville,0
4,AL - Mobile,0


In [101]:
#getting the total sum of occurance
Medicare_data_REGION_GROUPED = pd.DataFrame(Medicare_data_REGION[['Hospital Referral Region (HRR) Description','MAX_sum']].groupby(['Hospital Referral Region (HRR) Description']).agg('sum').add_suffix('_agg'),)

In [104]:
Medicare_data_REGION_GROUPED['MAX_sum_agg'].nlargest(3)

Hospital Referral Region (HRR) Description
CA - Contra Costa County    36
CA - San Mateo County       24
CA - Santa Cruz             11
Name: MAX_sum_agg, dtype: int64

    #5. PART1D - Largest claim difference for the largest number of procedures by providers

    We see largest claim difference for highest number of procedures for InpatientDGR for the follwing providers -
    
    BAYONNE HOSPITAL CENTER, NJ          29
    CROZER CHESTER MEDICAL CENTER, PA    12
    HAHNEMANN UNIVERSITY HOSPITAL, PA     8

#### InpatientDRG

    Considering -
    Average Covered Charges as Claims
    Average Total Payments as Reimbursement

In [92]:
Medicare_Provider_Charge_In.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region (HRR) Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,32963.07692,5777.241758,4763.736264
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,15131.85714,5787.571429,4976.714286
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,37560.37500,5434.958333,4453.791667
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,13998.28000,5417.560000,4129.160000
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,31633.27778,5658.333333,4851.444444


In [93]:
Medicare_IN_DRG = Medicare_Provider_Charge_In[['DRG Definition', 'Provider Name','Average Total Payments', 'Average Covered Charges' ]]

In [94]:
#Difference of claim and reimbursement
Medicare_IN_DRG['CLAIM_DIFF'] = Medicare_IN_DRG.apply(lambda x: x['Average Covered Charges'] - x['Average Total Payments'],axis =1)

C:\Users\lenovo\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [95]:
Medicare_IN_DRG.head(5)

,DRG Definition,Provider Name,Average Total Payments,Average Covered Charges,CLAIM_DIFF
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,SOUTHEAST ALABAMA MEDICAL CENTER,5777.241758,32963.07692,27185.835162
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,MARSHALL MEDICAL CENTER SOUTH,5787.571429,15131.85714,9344.285711
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ELIZA COFFEE MEMORIAL HOSPITAL,5434.958333,37560.37500,32125.416667
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ST VINCENT'S EAST,5417.560000,13998.28000,8580.720000
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,SHELBY BAPTIST MEDICAL CENTER,5658.333333,31633.27778,25974.944447


In [96]:
#Finding the mean of Average Covered Charges for InpatientDRG by DRG Defination and Provider
Medicare_IN_AGG = pd.DataFrame(Medicare_IN_DRG[['DRG Definition', 'Provider Name', 'CLAIM_DIFF']].groupby(['DRG Definition', 'Provider Name']).agg('mean').add_suffix('_avg'))

In [97]:
Medicare_IN_AGG = Medicare_IN_AGG.reset_index()

In [98]:
Medicare_IN_AGG .head()

,DRG Definition,Provider Name,CLAIM_DIFF_avg
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ABBOTT NORTHWESTERN HOSPITAL,27026.166671
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ABILENE REGIONAL MEDICAL CENTER,29293.857146
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ABINGTON MEMORIAL HOSPITAL,33207.235299
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ADENA REGIONAL MEDICAL CENTER,16247.499998
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ADVENTIST LA GRANGE MEMORIAL HOSPITAL,30415.357141


In [99]:
#Finding the max for InpatientDRG DRG Definition
Medicare_IN_MAX = pd.DataFrame(Medicare_IN_AGG[['DRG Definition', 'CLAIM_DIFF_avg']].groupby(['DRG Definition']).agg('max').add_suffix('_max'))

In [100]:
Medicare_IN_MAX = Medicare_IN_MAX.reset_index()

In [101]:
Medicare_IN_MAX.head()

,DRG Definition,CLAIM_DIFF_avg_max
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,135377.61538
1,057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/...,122609.00000
2,064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,214346.92103
3,065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,144842.08333
4,066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,109535.14290


In [102]:
#Merging The mean and max - will give side by side comparison of region having highest claim difference
Medicare_IN_MERGE = Medicare_IN_AGG.merge(Medicare_IN_MAX,on='DRG Definition', how='left')

In [103]:
Medicare_IN_MERGE = Medicare_IN_MERGE.reset_index()

In [104]:
Medicare_IN_MERGE.head()

,index,DRG Definition,Provider Name,CLAIM_DIFF_avg,CLAIM_DIFF_avg_max
0,0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ABBOTT NORTHWESTERN HOSPITAL,27026.166671,135377.61538
1,1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ABILENE REGIONAL MEDICAL CENTER,29293.857146,135377.61538
2,2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ABINGTON MEMORIAL HOSPITAL,33207.235299,135377.61538
3,3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ADENA REGIONAL MEDICAL CENTER,16247.499998,135377.61538
4,4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,ADVENTIST LA GRANGE MEMORIAL HOSPITAL,30415.357141,135377.61538


In [105]:
#function to get the max indicator
def difftyp(Medicare_IN_MERGE):
    if ((Medicare_IN_MERGE['CLAIM_DIFF_avg'] == Medicare_IN_MERGE['CLAIM_DIFF_avg_max'])):
        return 1
    elif ((Medicare_IN_MERGE['CLAIM_DIFF_avg'] > Medicare_IN_MERGE['CLAIM_DIFF_avg_max'])):
        return 1
    else:
        return 0

In [106]:
Medicare_IN_MERGE['MAX'] = Medicare_IN_MERGE.apply(difftyp, axis=1)

In [107]:
Medicare_IN_MERGE[Medicare_IN_MERGE['MAX'] == 1].head()

,index,DRG Definition,Provider Name,CLAIM_DIFF_avg,CLAIM_DIFF_avg_max,MAX
980,980,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,UNIVERSITY OF CALIFORNIA DAVIS MEDICAL CENTER,135377.61538,135377.61538,1
2048,2048,057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/...,STANFORD HOSPITAL,122609.00000,122609.00000,1
2533,2533,064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,DOCTORS MEDICAL CENTER,214346.92103,214346.92103,1
3953,3953,065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,BAYONNE HOSPITAL CENTER,144842.08333,144842.08333,1
6514,6514,066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,HAHNEMANN UNIVERSITY HOSPITAL,109535.14290,109535.14290,1


In [108]:
#getting number of times a provider has max claim diff for max procudures
Medicare_IN_MERGE_GROUPED = pd.DataFrame(Medicare_IN_MERGE[['Provider Name','MAX']].groupby(['Provider Name']).agg('sum').add_suffix('_sum'),)

In [109]:
Medicare_IN_MERGE_GROUPED['MAX_sum'].nlargest(3)

Provider Name
BAYONNE HOSPITAL CENTER          29
CROZER CHESTER MEDICAL CENTER    12
HAHNEMANN UNIVERSITY HOSPITAL     8
Name: MAX_sum, dtype: int64

#### OutPatientsAPC

    Considering -
    Average Estimated Submitted Charges as Claims
    Average Total Payments as Reimbursement

In [129]:
Medicare_Provider_Charge_Out.head()

,APC,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region (HRR) Description,Outpatient Services,Average Estimated Submitted Charges,Average Total Payments
0,0012 - Level I Debridement & Destruction,10029,EAST ALABAMA MEDICAL CENTER AND SNF,2000 PEPPERELL PARKWAY,OPELIKA,AL,36801,AL - Birmingham,23,78.086957,21.910435
1,0012 - Level I Debridement & Destruction,20024,CENTRAL PENINSULA GENERAL HOSPITAL,250 HOSPITAL PLACE,SOLDOTNA,AK,99669,AK - Anchorage,994,149.589749,36.623853
2,0012 - Level I Debridement & Destruction,30064,"UNIVERSITY OF ARIZONA MEDICAL CTR-UNIVERSIT, THE",1501 NORTH CAMPBELL AVENUE,TUCSON,AZ,85724,AZ - Tucson,1765,50.135411,14.541841
3,0012 - Level I Debridement & Destruction,30088,BANNER BAYWOOD MEDICAL CENTER,6644 EAST BAYWOOD AVENUE,MESA,AZ,85206,AZ - Mesa,20,112.400000,23.736000
4,0012 - Level I Debridement & Destruction,30111,"UNIVERSITY OF ARIZONA MEDICAL CTR- SOUTH CAM, THE",2800 EAST AJO WAY,TUCSON,AZ,85713,AZ - Tucson,22,152.045455,16.569091


In [130]:
Medicare_Provider_Charge_Out.columns

Index([u'APC', u'Provider Id', u'Provider Name', u'Provider Street Address',
       u'Provider City', u'Provider State', u'Provider Zip Code',
       u'Hospital Referral Region (HRR) Description', u'Outpatient Services',
       u'Average  Estimated Submitted Charges', u'Average Total Payments'],
      dtype='object')

In [131]:
Medicare_Out_DRG = Medicare_Provider_Charge_Out[['APC', 'Provider Name', 'Average  Estimated Submitted Charges', 'Average Total Payments' ]]

In [132]:
#Difference of claim and reimbursement
Medicare_Out_DRG['CLAIM_DIFF'] = Medicare_Out_DRG.apply(lambda x: x['Average  Estimated Submitted Charges'] - x['Average Total Payments'],axis =1)

C:\Users\lenovo\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [133]:
Medicare_Out_DRG.head(1)

,APC,Provider Name,Average Estimated Submitted Charges,Average Total Payments,CLAIM_DIFF
0,0012 - Level I Debridement & Destruction,EAST ALABAMA MEDICAL CENTER AND SNF,78.086957,21.910435,56.176522


In [134]:
#Finding the mean of Average Covered Charges for InpatientDRG by DRG Defination and Provider
Medicare_Out_AGG = pd.DataFrame(Medicare_Out_DRG[['APC', 'Provider Name', 'CLAIM_DIFF']].groupby(['APC', 'Provider Name']).agg('mean').add_suffix('_avg'))

In [135]:
Medicare_Out_AGG = Medicare_Out_AGG.reset_index()

In [136]:
Medicare_Out_AGG .head()

,APC,Provider Name,CLAIM_DIFF_avg
0,0012 - Level I Debridement & Destruction,ALTON MEMORIAL HOSPITAL,91.856818
1,0012 - Level I Debridement & Destruction,ALTRU HOSPITAL,7.259495
2,0012 - Level I Debridement & Destruction,ARNOT OGDEN MEDICAL CENTER,98.893846
3,0012 - Level I Debridement & Destruction,"AROOSTOOK MEDICAL CENTER,THE",16.975013
4,0012 - Level I Debridement & Destruction,AVERA MCKENNAN HOSPITAL & UNIVERSITY HEALTH CE...,22.306715


In [137]:
#Finding the max for InpatientDRG DRG Definition
Medicare_Out_MAX = pd.DataFrame(Medicare_Out_AGG[['APC', 'CLAIM_DIFF_avg']].groupby(['APC']).agg('max').add_suffix('_max'))

In [138]:
Medicare_Out_MAX = Medicare_Out_MAX.reset_index()

In [139]:
Medicare_Out_MAX.head()

,APC,CLAIM_DIFF_avg_max
0,0012 - Level I Debridement & Destruction,827.44000
1,0013 - Level II Debridement & Destruction,5175.40000
2,0015 - Level III Debridement & Destruction,2980.73963
3,0019 - Level I Excision/ Biopsy,26073.80500
4,0020 - Level II Excision/ Biopsy,18362.75882


In [140]:
#Merging The mean and max - will give side by side comparison of region having highest claim difference
Medicare_Out_MERGE = Medicare_Out_AGG.merge(Medicare_Out_MAX,on='APC', how='left')

In [141]:
Medicare_Out_MERGE = Medicare_Out_MERGE.reset_index()

In [142]:
Medicare_Out_MERGE.head()

,index,APC,Provider Name,CLAIM_DIFF_avg,CLAIM_DIFF_avg_max
0,0,0012 - Level I Debridement & Destruction,ALTON MEMORIAL HOSPITAL,91.856818,827.44
1,1,0012 - Level I Debridement & Destruction,ALTRU HOSPITAL,7.259495,827.44
2,2,0012 - Level I Debridement & Destruction,ARNOT OGDEN MEDICAL CENTER,98.893846,827.44
3,3,0012 - Level I Debridement & Destruction,"AROOSTOOK MEDICAL CENTER,THE",16.975013,827.44
4,4,0012 - Level I Debridement & Destruction,AVERA MCKENNAN HOSPITAL & UNIVERSITY HEALTH CE...,22.306715,827.44


In [143]:
#function to get the max indicator
def difftyp(Medicare_Out_MERGE):
    if ((Medicare_Out_MERGE['CLAIM_DIFF_avg'] == Medicare_Out_MERGE['CLAIM_DIFF_avg_max'])):
        return 1
    elif ((Medicare_Out_MERGE['CLAIM_DIFF_avg'] > Medicare_Out_MERGE['CLAIM_DIFF_avg_max'])):
        return 1
    else:
        return 0

In [144]:
Medicare_Out_MERGE['MAX'] = Medicare_Out_MERGE.apply(difftyp, axis=1)

In [145]:
Medicare_Out_MERGE[Medicare_Out_MERGE['MAX'] == 1].head()

,index,APC,Provider Name,CLAIM_DIFF_avg,CLAIM_DIFF_avg_max,MAX
152,152,0012 - Level I Debridement & Destruction,NEBRASKA ORTHOPAEDIC HOSPITAL,827.44000,827.44000,1
1265,1265,0013 - Level II Debridement & Destruction,RUSSELLVILLE HOSPITAL,5175.40000,5175.40000,1
2703,2703,0015 - Level III Debridement & Destruction,RESEARCH MEDICAL CENTER,2980.73963,2980.73963,1
3247,3247,0019 - Level I Excision/ Biopsy,CENTINELA HOSPITAL MEDICAL CENTER,26073.80500,26073.80500,1
4255,4255,0020 - Level II Excision/ Biopsy,ORANGE COAST MEMORIAL MEDICAL CENTER,18362.75882,18362.75882,1


In [146]:

#getting number of times a provider has max claim diff for max procudures
Medicare_Out_MERGE_GROUPED = pd.DataFrame(Medicare_Out_MERGE[['Provider Name','MAX']].groupby(['Provider Name']).agg('sum').add_suffix('_sum'))

In [159]:
Medicare_Out_MERGE_GROUPED = Medicare_Out_MERGE_GROUPED.reset_index()

In [160]:
Medicare_Out_MERGE_GROUPED.head()

,Provider Name,MAX_sum
0,ABBEVILLE GENERAL HOSPITAL,0
1,ABBOTT NORTHWESTERN HOSPITAL,0
2,ABILENE REGIONAL MEDICAL CENTER,0
3,ABINGTON MEMORIAL HOSPITAL,0
4,ACMH HOSPITAL,0


In [148]:
Medicare_Out_MERGE_GROUPED['MAX_sum'].nlargest(3)

Provider Name
CAPITAL HEALTH MEDICAL CENTER - HOPEWELL    2
COLUMBIA HOSPITAL                           2
FORT WALTON BEACH MEDICAL CENTER            2
Name: MAX_sum, dtype: int64

    ****************************************** END OF PART 1 *******************************************************